# Playing with RNN

In [3]:

%load_ext autoreload
%autoreload 2
import os
import tensorflow as tf
from src import TextDataset, TextGeneration, RNN, LSTM, Sampling, Config

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Reading TextDataset

In [4]:
config = {
    'DATA': {
        'WINDOW_SIZE': 35,
        'STRIDE': 3,
        'DATA_PATH': os.path.join("data", "beatles.txt"),
    },
    "TRAINING": {
        "HIDDEN_STATE_SIZE": [256, 512],
        "BATCH_SIZE": 2500,
        "EPOCHS": 250,
    },
}
config = Config(**config)

DataConfig(WINDOW_SIZE=35, STRIDE=3, DATA_PATH='data/beatles.txt')
TrainingConfig(HIDDEN_STATE_SIZE=[256, 512], EPOCHS=250, BATCH_SIZE=2500, DROPOUT=0.5, LR=0.001, BUFFER_SIZE=10000, SAVE_DIR='models', PRED_EVERY=5, PRED_LEN=100, PRED_TEMP=0.75)


In [5]:

data = TextDataset(config, verbosity=1).read()
print(data)

# [TextDataset]:	Config(DATA=DataConfig(WINDOW_SIZE=35, STRIDE=3, DATA_PATH='data/beatles.txt'), TRAINING=TrainingConfig(HIDDEN_STATE_SIZE=[256, 512], EPOCHS=250, BATCH_SIZE=2500, DROPOUT=0.5, LR=0.001, BUFFER_SIZE=10000, SAVE_DIR='models', PRED_EVERY=5, PRED_LEN=100, PRED_TEMP=0.75), EMBED=None)
	166753 total chars in text
# [Embeddings - init]:	label=beatles	encoding=utf-8
# Dateset Summary [label=`beatles`]:
=> Configuration:
	DataConfig(WINDOW_SIZE=35, STRIDE=3, DATA_PATH='data/beatles.txt')
=> 10 Samples:
	1. b'a day in the life\ni read the news t' => b' day in the life\ni read the news to'
	2. b'ay in the life\ni read the news toda' => b'y in the life\ni read the news today'
	3. b'in the life\ni read the news today o' => b'n the life\ni read the news today oh'
	4. b'the life\ni read the news today oh b' => b'he life\ni read the news today oh bo'
	5. b' life\ni read the news today oh boy\n' => b'life\ni read the news today oh boy\na'
	6. b'fe\ni read the news today oh boy\nabo' =>

### Peek at the One-Hot Encoded Data

In [4]:
print(data.dataset_oh)
for i in data.dataset_oh.take(5):
    print(i)

<MapDataset shapes: ((35, 49), (35, 49)), types: (tf.int64, tf.int64)>
(<tf.Tensor: shape=(35, 49), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, <tf.Tensor: shape=(35, 49), dtype=int64, numpy=
array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>)
(<tf.Tensor: shape=(35, 49), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, <tf.Tensor: shape=(35, 49), dtype=int64, numpy=
array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
      

## Generating Text - Dry Run

In [6]:
generator = TextGeneration(config=config, model_func=LSTM.get_LSTM, verbosity=1)

# [TextGeneration]:	model_func=get_LSTM	verbosity=1


In [ ]:
generator.train(data, config.TRAINING)

In [7]:
# Load model from models/beatles/model_LSTM_beatles_WS-35_ST-3_BS-2500_HS-256-512_DR-0.5_LR-0.001
generator.load_model(os.path.join("models", "beatles"), 0)

ValueError: No models found in models/beatles

In [74]:
tf.keras.utils.set_random_seed(0)
prompt = "i swear, man, this ain't"
print("=" * 80)
generated = generator.predict(prompt, sampling_method=Sampling.greedy_search, pred_len=500,
                              include_prompt=True)
print("=" * 80)
print(generated)

# [TextGeneration - predict]:	prompt_len=24	num_chars=500
Old Function


Generating...: 100%|██████████| 500/500 [00:17<00:00, 28.50it/s, char=[b'u']]           

i swear, man, this ain't got nothing to say but it’s o.k.
good morning, good morning...
people running round it's five o’clock.
everywhere in town with you
i don’t know why you say goodbye, i say hello
hello hello
i don't know why you say goodbye, i say hello
hello hello
i don't know why you say goodbye, i say hello
hello hello
i don't know why you say goodbye, i say hello
hello hello
i don't know why you say goodbye, i say hello
hello hello
i don't know why you say goodbye, i say hello
hello hello
i don't know why you


In [75]:
tf.keras.utils.set_random_seed(0)
prompt = "i swear, man, this ain't"
print("=" * 80)
generated = generator.predict(prompt, sampling_method=Sampling.random_sampling_bugged, pred_len=500,
                              include_prompt=True)
print("=" * 80)
print(generated)

# [TextGeneration - predict]:	prompt_len=24	num_chars=500
Old Function


Generating...: 100%|██████████| 500/500 [00:17<00:00, 28.48it/s, char=[b'e']]           

i swear, man, this ain't no dancer.
look out helter skelter
helter skelter
look out helter skelter
helter skelter
let me have to say that i’ve been unhappy with you
i see no day
if i till your feet.
roll up to make a doe who is all belong?
sit beside a mountain waiting for this moment to arise
you were only waiting for this moment to arise.
blackbird singing in the dead of night
take these broken wings and learn to see
all the cold and to you.
all my loving, darling
if you wanna dance with me
if you wanna dance with me


In [78]:
tf.keras.utils.set_random_seed(0)
prompt = "i swear, man, this ain't"
print("=" * 80)
generated = generator.predict(prompt, sampling_method=Sampling.random_sampling, pred_len=500,
                              temp=0.75,
                              include_prompt=True)
print("=" * 80)
print(generated)

# [TextGeneration - predict]:	prompt_len=24	num_chars=500
Old Function


Generating...: 100%|██████████| 500/500 [00:18<00:00, 27.68it/s, char=[b' ']]           

i swear, man, this ain't the kind that you can see.
whoa, oh, these chains of love got a hold on me.
believe me when i tell you
i’m looking through you, you’re never weep at night i said these words to my girl,
i have got another man
that’s the end’a little girl
help you should see polythene pam
you better run for your life if you can, little girl
and why you lied to me,
tell me why
you can send it here
but i never heard it singing
no i never heard it at all.
would it out of my window
you’re coming home, you’re coming 
